In [ ]:
!pip install anomalib
!anomalib install

In [8]:
# Import the required modules
from anomalib.models import Patchcore, Padim, Stfpm, Cflow, Dfm, Dfkde, Dsr, Fastflow, Fre, Ganomaly, Uflow, Rkde
from anomalib.engine import Engine
from anomalib.data import Folder
from anomalib.metrics import *
from anomalib.loggers import AnomalibTensorBoardLogger
import matplotlib.pyplot as plt
import os, copy

# Train Data module
train_datamodule = Folder(
    name="GlassDefect",
    root="/content/drive/MyDrive/Simple-AI-Task/GlassDefect",
    normal_dir="normal_train",
    abnormal_dir="abnormal_train",
    task="classification",
    image_size=(128,128),
    num_workers=1,
)
train_datamodule.setup()

# Test Data module
test_datamodule = Folder(
    name="GlassDefect",
    root="/content/drive/MyDrive/Simple-AI-Task/GlassDefect",
    normal_dir="normal_test",
    abnormal_dir="abnormal_test",
    task="classification",
    image_size=(128,128),
    num_workers=1,
)
test_datamodule.setup()


# draw curves
def draw_graphs(ground_truth, predicted):
    metric = AUPR()
    metric.update(predicted, ground_truth)
    metric.compute()
    figure1, title1 = metric.generate_figure()
    plt.savefig(f"/content/drive/MyDrive/Simple-AI-Task/results/{type(model).__name__}/AUPR.png")

    metric = AUROC()
    metric.update(predicted, ground_truth)
    metric.compute()
    figure2, title2 = metric.generate_figure()
    plt.savefig(f"/content/drive/MyDrive/Simple-AI-Task/results/{type(model).__name__}/AUROC.png")

    metric = AUPRO()
    metric.update(predicted, ground_truth)
    metric.compute()
    figure2, title2 = metric.generate_figure()
    plt.savefig(f"/content/drive/MyDrive/Simple-AI-Task/results/{type(model).__name__}/AUPRO.png")


# test and train models
def training(model, train_datamodule, test_datamodule, ckpt):
    logger = AnomalibTensorBoardLogger("tb_logs", name=f"{type(model).__name__}")
    engine = Engine(
        max_epochs=20,
        logger=logger,
    )
    # train model
    engine.fit(datamodule=train_datamodule, model=model)
    # test model
    tests = engine.test(
        datamodule=test_datamodule,
        model=model,
        ckpt_path=ckpt,
    )

# inference using models
def inference(model, test_datamodule, ckpt):
    logger = AnomalibTensorBoardLogger("/content/drive/MyDrive/Simple-AI-Task/results", name=f"{type(model).__name__}")
    engine = Engine(
        logger=logger,
    )
    predictions = engine.predict(
        datamodule=test_datamodule,
        model=model,
        ckpt_path=ckpt,
        return_predictions=True,
    )
    draw_graphs(predicted=predictions[0]['pred_scores'], ground_truth=predictions[0]['label'])

# draw inferred images
def plot_images(folder_path):
    # Get a list of all the files in the folder
    file_list = os.listdir(folder_path)

    # Filter the list to only include image files
    image_files = [f for f in file_list if f.endswith('.png')]

    # Calculate the number of rows needed
    num_images = len(image_files)
    num_rows = (num_images + 1) // 2  # Round up to the nearest integer

    # Create a figure with the appropriate number of rows and columns
    fig, axes = plt.subplots(num_rows, 2, figsize=(10, 2 * num_rows))

    # Loop through the images and display them
    for i, image_file in enumerate(image_files):
        row = i // 2
        col = i % 2
        image_path = os.path.join(folder_path, image_file)
        axes[row, col].imshow(plt.imread(image_path))
        axes[row, col].axis('off')

    # Adjust the spacing between subplots
    plt.subplots_adjust(wspace=0.1, hspace=0.1)

    # Display the figure
    plt.show()

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Batch Training

In [4]:
model_list = [Patchcore(), Padim(), Stfpm(), Dfm(), Fastflow(), Rkde(),]
for model in model_list: training(model, copy.deepcopy(train_datamodule), copy.deepcopy(test_datamodule), f"results/{type(model).__name__}/GlassDefect/latest/weights/lightning/model.ckpt")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
INFO: Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:lightning.pytorch.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.rich_model_summary.RichM

┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name                  ┃ Type                     ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ model                 │ PatchcoreModel           │ 24.9 M │
│ 1 │ _transform            │ Compose                  │      0 │
│ 2 │ normalization_metrics │ MinMax                   │      0 │
│ 3 │ image_threshold       │ F1AdaptiveThreshold      │      0 │
│ 4 │ pixel_threshold       │ F1AdaptiveThreshold      │      0 │
│ 5 │ image_metrics         │ AnomalibMetricCollection │      0 │
│ 6 │ pixel_metrics         │ AnomalibMetricCollection │      0 │
└───┴───────────────────────┴──────────────────────────┴────────┘

Trainable params: 24.9 M                                                                                           
Non-trainable params: 0                                                                                            
Total params: 24.9 M                                                                                               
Total estimated model params size (MB): 99

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (4) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Output()

...

INFO: Restoring states from the checkpoint path at /content/results/Rkde/GlassDefect/v2/weights/lightning/model.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Restoring states from the checkpoint path at /content/results/Rkde/GlassDefect/v2/weights/lightning/model.ckpt
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: Loaded model weights from the checkpoint at /content/results/Rkde/GlassDefect/v2/weights/lightning/model.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Loaded model weights from the checkpoint at /content/results/Rkde/GlassDefect/v2/weights/lightning/model.ckpt


Output()

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        image_AUROC        │            1.0            │
│       image_F1Score       │            1.0            │
└───────────────────────────┴───────────────────────────┘

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


#### Training Curves

In [9]:
for model in model_list: inference(model, copy.deepcopy(test_datamodule), f"results/{type(model).__name__}/GlassDefect/latest/weights/lightning/model.ckpt")

INFO: Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:lightning.pytorch.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: Restoring states from the checkpoint path at /content/results/Patchcore/G

Output()

INFO: Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:lightning.pytorch.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: `Trainer(val_check_interval=1.0)` was configured so validation will run a

Output()

INFO: Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:lightning.pytorch.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: Restoring states from the checkpoint path at /content/results/Stfpm/Glass

...

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


/usr/local/lib/python3.10/dist-packages/anomalib/utils/visualization/image.py:316: RuntimeWarning: More than 20 
figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained 
until explicitly closed and may consume too much memory. (To control this warning, see the rcParam 
`figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  self.figure, self.axis = plt.subplots(1, num_cols, figsize=figure_size)

/usr/local/lib/python3.10/dist-packages/anomalib/metrics/plotting_utils.py:47: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, axis = plt.subplots()
INFO: Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:lightning.pytorch.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, usi

Output()

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


#### TensorBoard Results

Run the code below inside console to get all the results inside the tensorboard dashboard which is running by default on http://localhost:6006/

In [1]:
!tensorboard --logdir="Simple-AI-Task-20240612T105804Z-001/Simple-AI-Task/tb_logs"

^C
